In [1]:
import torch
import torch.nn as nn
import transformers
from peft import (
    PeftConfig,
    PeftModel,
    get_peft_model
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

c:\Users\prath\anaconda3\envs\matr\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PEFT_MODEL = "prtm/ConstructionAdviserModelDraft1"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit = True

)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Users\prath\anaconda3\envs\matr\lib\site-packages\transformers\quantizers\auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [6]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Outline the breach associated with the given defect.", # instruction
        "The mechanical AC condenser has not been fixed into position as required by the manufacturer. Refrigerant pipes are rigid and can break leaking refrigerant gas into the atmosphere.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True)
outputs = tokenizer.batch_decode(outputs)

response_index = outputs[0].find("### Response:")
response = outputs[0][response_index + len("### Response:"):].strip()

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The condenser shall be bolted to the base plate as per manufacturer's specifications.

Failure to perform work with due care and skill – section 18B(1)(a) of the HBA.

Works have not been completed as per plans and specifications (manufacturer's specification) – section 18B(1)(a) of the HBA.

Materials not fit for purpose – section 18B(1)(f) of the HBA.  

Works have not been
